# **Fake News Detection using Passive-Aggressive Classifier and TF-IDF**
## **Introduction**
In today's digital age, the spread of misinformation and fake news has become a significant challenge. To combat this, we can leverage **Machine Learning** techniques to build a model that can classify news articles as **Real** or **Fake**.

In this notebook, we will:
- Convert news text into numerical features using **TF-IDF (Term Frequency-Inverse Document Frequency)**.
- Use a **Passive-Aggressive Classifier (PAC)**, an efficient online learning algorithm, to detect fake news.
- Evaluate model performance using **accuracy and F1-score**.

## **Why Passive-Aggressive Classifier?**
The **Passive-Aggressive Classifier** is well-suited for **real-time text classification** because:
- It **updates only when a mistake is made** (Passive when correct, Aggressive when incorrect).
- It works efficiently on **large, high-dimensional datasets** (like text data).
- It is a **fast and lightweight algorithm** suitable for news classification.


In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/real-or-fake-news/news.csv


### Import necessary library

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import f1_score


### Preprocessing

In [3]:
#Read the data
df = pd.read_csv('/kaggle/input/real-or-fake-news/news.csv')

#shape and head
df.shape
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
#labels
labels = df.label
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [5]:
#Spitting into test and train
x_train, x_test, y_train, y_test = train_test_split(df['text'], labels, test_size=0.2, random_state=8)

### Tfidf Vectorizer
	• Extracts important words in news articles.
	• Helps classify news as REAL or FAKE.
	• Used with Passive-Aggressive Classifier (PAC) for fast and effective classification.

In [6]:
#initialise a Tfidfvectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)

# Fit and transform 
tfidf_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_test = tfidf_vectorizer.transform(x_test)


### PAC algorithm
    1. Initialize a weight vector w (starting with zeros or small random values).
	2. Make a prediction using: y_pred = sign(w * x)
	3. If prediction is correct, do nothing (Passive).
	4. If incorrect, update w aggressively to correct the mistake (Aggressive): w = w + τyx

In [7]:
#initialise PassiveAgressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

x_pred = pac.predict(tfidf_train)
score_train = accuracy_score(y_train, x_pred)
print(f'Accuracy(Train): {score_train*100}%')

#Predict on test and calculate accurracy
y_pred = pac.predict(tfidf_test)
score  = accuracy_score(y_test, y_pred)
print(f'Accuracy(Test): {round(score*100)}%')

Accuracy(Train): 100.0%
Accuracy(Test): 94%


In [8]:
#metrics
f1 = f1_score(y_test, y_pred, pos_label='FAKE')
conf = confusion_matrix(y_test, y_pred, labels = ['FAKE', 'REAL'])
print(f'Recall: {f1:.3f}\n\nConfusion Matrix:\n{conf}')


Recall: 0.938

Confusion Matrix:
[[600  36]
 [ 44 587]]
